## Create a dask-gateway cluster from an Argo workflow

This notebook creates a small dask-gateway cluster.

It is a companion to `workflows/easi-dask-gateway.yaml`.

The pod/worker environment and parameters can be passed in from the Argo workflow.

In [ ]:
from dask_gateway import Gateway
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
def create_cluster(
    gateway: Gateway,
    cluster_name: str = None,
    new_cluster: bool: True,
    workers: tuple = (1,1),
    adapt = False,
    options = None,
):
    clusters = gateway.list_clusters()
    if cluster_name is not None and cluster_name in clusters:
        # Connect to existing cluster
        cluster = gateway.connect(cluster_name)   # This will block until the cluster enters the running state
        logging.info(f'Connected to existing cluster: {cluster_name}')
    else:
        # Start new cluster
        cluster_name = gateway.submit(cluster_options=options)   # This is non-blocking, multiple clusters can be requested simultaneously
        cluster = gateway.connect(cluster_name)
        logging.info(f'Created new cluster: {cluster_name}')
    if adapt:
        logging.info('Set cluster to adapt')
        cluster.adapt(minimum = workers[0], maximum=workers[1])
    else:
        logging.info('Set cluster to scale')
        cluster.scale(workers[0])
    return cluster

In [ ]:
gateway = Gateway()
options = gateway.cluster_options()
cluster = create_cluster(gateway)
client = cluster.get_client()

# Show cluster and client objects and options
display(cluster)
display(client)
display(options)

In [ ]:
# Do work

In [ ]:
client.close()
cluster.shutdown()